```bash
$ pip install natasha
```

https://natasha.github.io/demo/

In [204]:
import natasha as ntsh
import os

In [206]:
# приведение файла с сущностями от к множеству сущностей
# каждая сущность - кортеж (тип, начало, конец, сущность)
def make_entities(file):
    # открыть, считать и разбить на строки (сущности) файл
    text = file.read().split('\n')
    # оставить в сущностях только их тип, начало, конец и строку с ней
    text = [line[line.index('\t')+1:].replace('\t', ' ') for line in text if line != '']
    # привести сущности из строк к кортежам
    entities = [entity.split(' ') for entity in text]
    entities = set([(entity[0], int(entity[1]), int(entity[2]), ' '.join(entity[3:])) for entity in entities])
    # получить отдельно сущности разных типов
    per_entities = set([entity for entity in entities if entity[0] == 'PER'])
    loc_entities = set([entity for entity in entities if entity[0] == 'LOC'])
    org_entities = set([entity for entity in entities if entity[0] == 'ORG'])
    return entities, per_entities, loc_entities, org_entities


In [207]:
def make_my_entities(text, matches, entities_type):
    entities = set()
    for match in matches:
        #print(match.span, match.fact)
        poses = []
        entity = ""
        for token in match.tokens:
            # оставляем в очередном токене только позицию и сущность
            ent, pos = [i for i in token][:2]
            # natasha учитывает \n как символ, а корпус - нет => нужен сдвиг позиции - d
            d = text[:pos[0]].count('\n')
            pos = [pos[0]+d, pos[1]+d]
            poses.append(pos)
            entity = entity + ent + ' '
        entity = (entities_type, poses[0][0], poses[-1][1], entity[:-1])
        entities.add(entity)
    return entities


In [208]:
# создание списка файлов корпуса для чтения
corpus = 'Collection3'
files = os.listdir(corpus)
files = [file[:-4] for file in files if '.txt' in file]

In [214]:
%%time
TP_per = FP_per = FN_per = 0
TP_loc = FP_loc = FN_loc = 0
TP_org = FP_org = FN_org = 0
TP = FP = FN = 0
for file in files:
    # октрытие и чтение файла
    text = open('Collection3/' + file + '.txt', 'r').read()
    # подготовленные в корпусе сущности из файла
    entities, per_entities, loc_entities, org_entities, = make_entities(open('Collection3/' + file + '.ann', 'r'))
    # экстракторы natasha
    per_extractor = ntsh.NamesExtractor()
    loc_extractor = ntsh.LocationExtractor()
    org_extractor = ntsh.OrganisationExtractor()
    # извлеченные сущности
    per_matches = per_extractor(text)
    loc_matches = loc_extractor(text)
    org_matches = org_extractor(text)
    # извлеченные сущности в нужном формате
    my_per_entities = make_my_entities(text, per_matches, "PER")
    my_loc_entities = make_my_entities(text, loc_matches, "LOC")
    my_org_entities = make_my_entities(text, org_matches, "ORG")
    my_entities = my_per_entities | my_loc_entities | my_org_entities
    
    TP_per += len(my_per_entities & per_entities)
    FP_per += len(my_per_entities - per_entities)
    FN_per += len(per_entities - my_per_entities)
    
    TP_org += len(my_org_entities & org_entities)
    FP_org += len(my_org_entities - org_entities)
    FN_org += len(org_entities - my_org_entities)
    
    TP_loc += len(my_loc_entities & loc_entities)
    FP_loc += len(my_loc_entities - loc_entities)
    FN_loc += len(loc_entities - my_loc_entities)
    
    TP += len(my_entities & entities)
    FP += len(my_entities - entities)
    FN += len(entities - my_entities)

CPU times: user 8min 41s, sys: 635 ms, total: 8min 42s
Wall time: 8min 42s


In [232]:
# функция для вычисления точности, полноты и F-меры

def prf(TP, FP, FN):
    P = float(TP)/(TP+FP)
    R = float(TP)/(TP+FN)
    F = 2*P*R/(P+R)
    return P, R, F

In [233]:
print(TP_per, FP_per, FN_per)
P, R, F = prf(TP_per, FP_per, FN_per)
print('Оценки для имен')
print('Точность:', P)
print('Полнота: ', R)
print('F-мера:  ', F)

6297 3741 4326
Оценки для имен
Точность: 0.6273161984459056
Полнота:  0.592770403840723
F-мера:   0.6095542326121679


In [234]:
print(TP_org, FP_org, FN_org)
P, R, F = prf(TP_org, FP_org, FN_org)
print('Оценки для организаций')
print('Точность:', P)
print('Полнота: ', R)
print('F-мера:  ', F)

1653 2503 6889
Оценки для организаций
Точность: 0.39773820981713187
Полнота:  0.19351439943807072
F-мера:   0.26035596156875096


In [235]:
print(TP_loc, FP_loc, FN_loc)
P, R, F = prf(TP_loc, FP_loc, FN_loc)
print('Оценки для географических названий')
print('Точность:', P)
print('Полнота: ', R)
print('F-мера:  ', F)

5237 3840 2006
Оценки для географических названий
Точность: 0.5769527376886636
Полнота:  0.7230429380091122
F-мера:   0.6417892156862745


In [236]:
print(TP, FP, FN)
P, R, F = prf(TP, FP, FN)
print('Общие оценки')
print('Точность:', P)
print('Полнота: ', R)
print('F-мера:  ', F)

13187 10084 13221
Общие оценки
Точность: 0.566670963860599
Полнота:  0.49935625568009695
F-мера:   0.5308883029046478
